In [1]:
import pandas as pd
import numpy as np

import os

import matplotlib.pyplot  as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import env

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

#### 1. Acquire data from the cloud database.
    You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.
    Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

In [2]:

def get_zillow():
    '''
    This function reads in zillow data from Codeup database using sql querry into a dataframe and returns a dataframe. 
    '''
        
    query = """
            SELECT * FROM 
            properties_2017
            left join predictions_2017 USING(parcelid)
            left join airconditioningtype USING(airconditioningtypeid)
            left join architecturalstyletype USING(architecturalstyletypeid)
            left join buildingclasstype USING(buildingclasstypeid)
            left join heatingorsystemtype USING(heatingorsystemtypeid)
            left join propertylandusetype USING(propertylandusetypeid)
            left join storytype USING(storytypeid)
            left join typeconstructiontype USING(typeconstructiontypeid)
            left join unique_properties USING(parcelid)
            where transactiondate like '2017%%';
            """
    
    filename = "zillow_2017.csv"
    
    # check if a file exits in a local drive
    if os.path.isfile(filename):
        
        #if yes, read data from a file
        df = pd.read_csv(filename)
        
    else:
        # If not, read data from database into a dataframe
        df = pd.read_sql(query, env.get_connection('zillow'))

        # Write that dataframe in to disk for later, Called "caching" the data for later.
        df.to_csv(filename, index=False)
     
    #returns dataframe
    return df

In [3]:
# acquire data using function
df = get_zillow()

# peak into data
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,3.5,4.0,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,None,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,None,NaN,6.059063e+13,0,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,1.0,2.0,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,None,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,None,NaN,6.111001e+13,1,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,None,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,None,NaN,6.059022e+13,2,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,None,NaN,6.037300e+13,3,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,3.0,3.0,8.0,3.0,NaN,NaN,1312.0,1312.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34185120.0,-118414640.0,278581.0,1.0,NaN,NaN,NaN,1.0,010C,LAR3,6.037124e+07,12447.0,3101.0,268548.0,96451.0,0.0,NaN,1.0,NaN,NaN,1964.0,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,None,NaN,6.037124e+13,4,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None


In [4]:
# get total rows and columns of data
df.shape

(77613, 69)

#### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [5]:
# Check for data types and count null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77613 entries, 0 to 77612
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77613 non-null  int64  
 1   typeconstructiontypeid        223 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77579 non-null  float64
 4   heatingorsystemtypeid         49571 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      207 non-null    float64
 7   airconditioningtypeid         25007 non-null  float64
 8   id                            77613 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77579 non-null  float64
 11  bedroomcnt                    77579 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c

#### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [6]:
df= df.sort_values('parcelid')

In [7]:
bool_id = df['parcelid'].duplicated()
bool_id.value_counts()

False    77414
True       199
Name: parcelid, dtype: int64

In [8]:
df = df[~bool_id]
df

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
55006,10711855,NaN,NaN,261.0,2.0,NaN,NaN,NaN,1087254,NaN,2.0,3.0,8.0,2.0,NaN,NaN,2107.0,2107.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,34222559.0,-118617387.0,9158.0,1.0,NaN,NaN,NaN,1.0,0101,LARE9,6.037113e+07,12447.0,3101.0,268588.0,96339.0,0.0,NaN,1.0,NaN,NaN,1972.0,NaN,NaN,249655.0,624139.0,2016.0,374484.0,7659.36,None,NaN,6.037113e+13,55006,-0.007357,2017-07-07,None,None,None,Central,Single Family Residential,None,None
71382,10711877,NaN,NaN,261.0,2.0,NaN,NaN,1.0,1072280,NaN,2.0,4.0,8.0,2.0,NaN,NaN,1882.0,1882.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,34220261.0,-118616409.0,9035.0,1.0,NaN,NaN,NaN,1.0,0101,LARE9,6.037113e+07,12447.0,3101.0,268588.0,96339.0,0.0,NaN,1.0,NaN,NaN,1972.0,NaN,NaN,253000.0,660000.0,2016.0,407000.0,8123.91,None,NaN,6.037113e+13,71382,0.021066,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
23209,10711888,NaN,NaN,261.0,2.0,NaN,NaN,1.0,1340933,NaN,2.0,4.0,8.0,2.0,NaN,NaN,1882.0,1882.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,34222491.0,-118616854.0,9800.0,NaN,NaN,NaN,NaN,NaN,0100,LARE9,6.037113e+07,12447.0,3101.0,268588.0,96339.0,0.0,NaN,1.0,NaN,NaN,1972.0,NaN,NaN,257591.0,542923.0,2016.0,285332.0,6673.24,None,NaN,6.037113e+13,23209,0.077174,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
18017,10711910,NaN,NaN,261.0,2.0,NaN,NaN,NaN,1878109,NaN,2.0,3.0,8.0,2.0,NaN,NaN,1477.0,1477.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,34221864.0,-118615739.0,11285.0,1.0,NaN,NaN,NaN,1.0,0101,LARE11,6.037113e+07,12447.0,3101.0,268588.0,96339.0,0.0,NaN,1.0,NaN,NaN,1960.0,NaN,NaN,57968.0,78031.0,2016.0,20063.0,1116.46,None,NaN,6.037113e+13,18017,-0.041238,2017-03-17,None,None,None,Central,Single Family Residential,None,None
20378,10711923,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2190858,NaN,2.0,4.0,8.0,2.0,NaN,NaN,1918.0,1918.0,NaN,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,34220619.0,-118615253.0,11239.0,1.0,NaN,NaN,NaN,1.0,0101,LARE11,6.037113e+07,12447.0,3101.0,268588.0,96339.0,0.0,NaN,1.0,NaN,NaN,1960.0,NaN,NaN,167869.0,415459.0,2016.0,247590.0,5239.85,None,NaN,6.037113e+13,20378,-0.009496,2017-03-24,None,None,None,Central,Single Family Residential,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13467,167686999,NaN,NaN,261.0,NaN,NaN,NaN,NaN,775695,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,34424104.0,-118468083.0,NaN,NaN,NaN,NaN,NaN,NaN,0100,SCRM,6.037920e+07,NaN,3101.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26405.0,2016.0,26405.0,988.48,None,NaN,NaN,13467,-0.068632,2017-02-28,None,None,None,None,Single Family Residential,None,None
14557,167687739,NaN,NaN,266.0,NaN,NaN,NaN,NaN,2863262,Na

In [9]:
df= df.sort_values('transactiondate')

In [10]:
bool_transactiondate1 = df['transactiondate'].duplicated(keep='first')
bool_transactiondate1 .value_counts()

# True     77349
# False      264

True     77150
False      264
Name: transactiondate, dtype: int64

In [11]:
bool_transactiondate2 = df['transactiondate'].duplicated(keep='last')
bool_transactiondate2 .value_counts()
# True     77349
# False      264

True     77150
False      264
Name: transactiondate, dtype: int64

In [12]:
# bool_transactiondate3 = df['logerror'].duplicated(keep='last')
# bool_transactiondate3 .value_counts()
# False    77064
# True       549

In [13]:
df = df[bool_transactiondate1]
df.shape

(77150, 69)

In [14]:
# df = df[~bool_transactiondate3]
# df.shape

In [15]:
boolean_1= ((pd.isnull(df['latitude'])) & (pd.isnull(df['longitude'])))

In [16]:
df= df[~boolean_1]
df.shape

(77117, 69)

In [17]:
df.latitude.isnull().sum()


0

In [18]:
df.longitude.isnull().sum()


0

In [19]:
df.longitude.isnull().sum()

0

In [20]:
df.shape

(77117, 69)

In [21]:
df.parcelid.isnull().sum()

0

In [22]:
df.airconditioningtypeid.isnull().sum()

52248

In [23]:
df.architecturalstyletypeid.isnull().sum()

76911

In [24]:
def row_missing(df):
    num_rows_missing = []
    percent_rows_missing = []
    index = df.columns.tolist()
    for col in df.columns:
        num_rows_missing.append(df[col].isnull().sum())
        percent_rows_missing.append(((df[col].isnull().sum())/len(df))* 100)
        
    metric_df = pd.DataFrame({'num_row_missing': num_rows_missing,'pct_rowss_missing':percent_rows_missing},index=index)
    return metric_df
        

In [25]:
nan_df = row_missing(df)
nan_df.head(10)

,num_row_missing,pct_rowss_missing
parcelid,0,0.0
typeconstructiontypeid,76895,99.712126
storytypeid,77067,99.935163
propertylandusetypeid,0,0.0
heatingorsystemtypeid,27851,36.115253
buildingclasstypeid,77102,99.980549
architecturalstyletypeid,76911,99.732873
airconditioningtypeid,52248,67.751598
id,id 0 id 0 dtype: int64,id 0.0 id 0.0 dtype: float64
basementsqft,77067,99.935163


#### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [26]:
df = df[df['propertylandusetypeid']== 261]
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,None,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,None,NaN,6.059022e+13,2,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,1.0,2.0,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,None,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,None,NaN,6.111001e+13,1,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
6,12095076,NaN,NaN,261.0,2.0,NaN,NaN,1.0,781532,NaN,3.0,4.0,9.0,3.0,NaN,NaN,2962.0,2962.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34145202.0,-118179824.0,63000.0,1.0,NaN,NaN,NaN,1.0,0101,PSR2,6.037461e+07,47019.0,3101.0,274684.0,96293.0,0.0,NaN,1.0,NaN,NaN,1950.0,NaN,NaN,276684.0,773303.0,2016.0,496619.0,9516.26,None,NaN,6.037461e+13,6,-0.001011,2017-01-01,Central,None,None,Central,Single Family Residential,None,None
7,12069064,NaN,NaN,261.0,NaN,NaN,NaN,NaN,870991,NaN,1.0,2.0,5.0,1.0,NaN,NaN,738.0,738.0,NaN,NaN,NaN,NaN,6037.0,NaN,1.0,NaN,NaN,NaN,34149214.0,-118239357.0,4214.0,NaN,NaN,NaN,NaN,NaN,0100,GLR4YY,6.037302e+07,45457.0,3101.0,276258.0,96325.0,0.0,NaN,1.0,NaN,NaN,1922.0,NaN,NaN,18890.0,218552.0,2016.0,199662.0,2366.08,None,NaN,6.037302e+13,7,0.101723,2017-01-01,None,None,None,None,Single Family Residential,None,None
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,3.5,4.0,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,None,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,None,NaN,6.059063e+13,0,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None


In [27]:
df.propertylandusetypeid.value_counts()

261.0    52137
Name: propertylandusetypeid, dtype: int64

#### 2. Create a function that will drop rows or columns based on the percent of values that are missing:

In [49]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    '''takes in a dataframe, proportion of required column and row, drop rows and columns above theahold
    return a dataframe'''
    threshold = int(round(prop_required_column*df.shape[0],0))
    print(threshold)
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*df.shape[1],0))
    print(threshold)
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df


In [48]:
handle_missing_values(df, prop_required_column = .75, prop_required_row = .75)

39103
22


,parcelid,propertylandusetypeid,id,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,logerror,transactiondate,propertylandusedesc
2,14186244,261.0,11677,2.0,3.0,2.0,1243.0,1243.0,6059.0,2.0,33886168.0,-117823170.0,8432.0,122,6.059022e+07,21412.0,1286.0,97078.0,6.0,1962.0,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,2,0.005383,2017-01-01,Single Family Residential
1,17052889,261.0,1387261,1.0,2.0,1.0,1465.0,1465.0,6111.0,1.0,34449266.0,-119281531.0,12647.0,1110,6.111001e+07,13091.0,2061.0,97099.0,5.0,1967.0,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,1,0.055619,2017-01-01,Single Family Residential
6,12095076,261.0,781532,3.0,4.0,3.0,2962.0,2962.0,6037.0,3.0,34145202.0,-118179824.0,63000.0,0101,6.037461e+07,47019.0,3101.0,96293.0,0.0,1950.0,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,6,-0.001011,2017-01-01,Single Family Residential
7,12069064,261.0,870991,1.0,2.0,1.0,738.0,738.0,6037.0,1.0,34149214.0,-118239357.0,4214.0,0100,6.037302e+07,45457.0,3101.0,96325.0,0.0,1922.0,18890.0,218552.0,2016.0,199662.0,2366.08,6.037302e+13,7,0.101723,2017-01-01,Single Family Residential
0,14297519,261.0,1727539,3.5,4.0,3.5,3100.0,3100.0,6059.0,3.0,33634931.0,-117869207.0,4506.0,122,6.059063e+07,53571.0,1286.0,96978.0,0.0,1998.0,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0,0.025595,2017-01-01,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77396,12070893,261.0,2466453,2.0,4.0,2.0,2474.0,2474.0,6037.0,2.0,34164976.0,-118250536.0,10740.0,0100,6.037301e+07,45457.0,3101.0,96326.0,0.0,1912.0,191516.0,838219.0,2016.0,646703.0,9170.82,6.037301e+13,77396,0.010359,2017-09-19,Single Family Residential
77398,12070929,261.0,1147540,2.0,3.0,2.0,3059.0,3059.0,6037.0,2.0,34166526.0,-118251456.0,14352.0,010G,6.037301e+07,45457.0,3101.0,96326.0,0.0,1915.0,158000.0,446000.0,2016.0,288000.0,4956.70,6.037301e+13,77398,0.021512,2017-09-19,Single Family Residential
77456,10897526,261.0,822314,2.0,3.0,2.0,1476.0,1476.0,6037.0,2.0,34167769.0,-118386088.0,5399.0,0100,6.037125e+07,12447.0,3101.0,96446.0,0.0,1947.0,134000.0,670000.0,2016.0,536000.0,8039.69,6.037125e+13,77456,0.066549,2017-09-19,Single Family Residential
77609,11000655,261.0,673515,2.0,2.0,2.0,1286.0,1286.0,6037.0,2.0,34245368.0,-118282383.0,47405.0,0100,6.037101e+07,12447.0,3101.0,96284.0,0.0,1940.0,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,77609,0.020615,2017-09-20,Single Family Residential


### Mall Customers


#### 1. Acquire data from the customers table in the mall_customers database.

In [28]:
# create sql querry

query = '''
        SELECT  * FROM 
        customers;
        '''

# acquire the data using sql querry
customers = pd.read_sql(query, env.get_connection('mall_customers'))

In [29]:
# peak into data

customers.head()

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


#### 2. Summarize the data (include distributions and descriptive statistics).

In [30]:
# check for data types and null values
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     200 non-null    int64 
 1   gender          200 non-null    object
 2   age             200 non-null    int64 
 3   annual_income   200 non-null    int64 
 4   spending_score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [31]:
# get statistical information
customers.describe()

,customer_id,age,annual_income,spending_score
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


#### 3. Detect outliers using IQR.

In [32]:
def detect_outliers(df):
    '''takes in a dataframe and print outliers of a dataframe'''
    for col in df.select_dtypes(exclude='object'):
        q1, q3 =df[col].quantile([.25, .75])
        iqr = q3 - q1 
        upper_bound = q3 + 1.5 * iqr
        lower_bound = q1- 1.5 * iqr
        print(col)
        print(f' upper bound: {upper_bound}, lower bound: {lower_bound}\n') 

In [33]:
detect_outliers(customers)

customer_id
 upper bound: 299.5, lower bound: -98.5

age
 upper bound: 79.375, lower bound: -1.625

annual_income
 upper bound: 132.75, lower bound: -13.25

spending_score
 upper bound: 130.375, lower bound: -22.625



#### 4. Split data into train, validate, and test.

In [34]:
def train_val_test(df, seed=42):
    '''split data into train, validate and test data'''
    
    # split data into 80% train_validate, 20% test
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed)
    
    # split train_validate data into 70% train, 30% validate
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed)
    
    # returns train, validate, test data
    return train, validate, test


In [35]:
# get shape of train, validate, and test data
train, validate, test = train_val_test(customers, seed=42)
train.shape, validate.shape, test.shape

((112, 5), (48, 5), (40, 5))

#### 5. Encode categorical columns using a one hot encoder (pd.get_dummies).

In [36]:
def make_dummies(df, target):
    '''takes a dataframe and target and return a dataframe with dummies'''
    dummies = pd.get_dummies(df[target],drop_first=True)
    df = pd.concat([df,dummies],axis=1)
    df = df.drop(columns=[target])
    return df
    

In [37]:
# get dataframe with encoded variables
train_encoded = make_dummies(train, 'gender')

# peak into dataframe
train_encoded.head()

,customer_id,age,annual_income,spending_score,Male
130,131,47,71,9,1
62,63,67,47,52,0
13,14,24,20,77,0
173,174,36,87,92,1
144,145,25,77,12,1


#### 6. Handles missing values.

In [38]:
# check for Null values
customers.isnull().sum()

customer_id       0
gender            0
age               0
annual_income     0
spending_score    0
dtype: int64

#### 7. Scaling

In [39]:
# create the scaler
scaler = MinMaxScaler()

#fit the scaler  and transform the columns
train_encoded[['age', 'annual_income']] = scaler.fit_transform(train_encoded[['age','annual_income']])


In [40]:
# peak into data
train_encoded.head()

,customer_id,age,annual_income,spending_score,Male
130,131,0.557692,0.459016,9,1
62,63,0.942308,0.262295,52,0
13,14,0.115385,0.040984,77,0
173,174,0.346154,0.590164,92,1
144,145,0.134615,0.508197,12,1


#### Encapsulate your work in a wrangle_mall.py python module.

In [41]:
def get_mall_customers():
    '''acquire mall customers data from Codeup database using sql querry into a dataframe and returns a dataframe. '''
    
    # create sql querry
    query = '''
            SELECT  * FROM 
            customers;
            '''

    # acquire the data using sql querry
    df = pd.read_sql(query, env.get_connection('mall_customers'))
    
    # returns a dataframe
    return df 
 
    
def detect_outliers(df):
    '''takes in a dataframe and print outliers of a dataframe'''
    for col in df.select_dtypes(exclude='object'):
        q1, q3 =df[col].quantile([.25, .75])
        iqr = q3 - q1 
        upper_bound = q3 + 1.5 * iqr
        lower_bound = q1- 1.5 * iqr
        print(col)
        print(f' upper bound: {upper_bound}, lower bound: {lower_bound}\n') 
        
        
def train_val_test(df, seed=42):
    '''split data into train, validate and test data'''
    
    # split data into 80% train_validate, 20% test
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed)
    
    # split train_validate data into 70% train, 30% validate
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed)
    
    # returns train, validate, test data
    return train, validate, test


def make_dummies(df, target):
    '''takes a dataframe and target and return a dataframe with dummies'''
     # create dummy vairable for categorical variables
    dummies = pd.get_dummies(df[target],drop_first=True)
    
    # concat dummy variables to dataframe
    df = pd.concat([df,dummies],axis=1)
    
    # drop repetative columns
    df = df.drop(columns=[target])
    
    # returns a dataframe
    return df



def wrangle_mall_customers() :
    '''This function acquire data, clean data, split data and returns train, vaidate and test data.'''
    
    # acquire mall customers data
    df = get_mall_customers()
    
    # get encoded mall customers data
    df = make_dummies(df, 'gender')
    
    # split data into train, validate and test data
    train, validate, test= train_val_test(df, seed=42)
    
    # returns train, validate, test data
    return train, validate, test
   

In [42]:
# get train, validate and test data
train, validate, test= wrangle_mall_customers()

# get shape of train, validate ans test data
train.shape, validate.shape, test.shape

((112, 5), (48, 5), (40, 5))

In [43]:
# peak into train data
train.head()

,customer_id,age,annual_income,spending_score,Male
130,131,47,71,9,1
62,63,67,47,52,0
13,14,24,20,77,0
173,174,36,87,92,1
144,145,25,77,12,1


In [44]:
def scale_data(train, validate, test, 
               columns_to_scale= ['age', 'annual_income'],
               return_scaler=False):
    ''' 
    Takes in train, validate, and test data and returns their scaled counterparts.
    If return_scalar is True, the scaler object will be returned as well
    '''
    # make copies of our original data so we dont gronk up anything
    train_scaled = train.copy()
    validate_scaled = validate.copy()
    test_scaled = test.copy()
    
    # use sacaler
    scaler = MinMaxScaler()
    
    # fit scaler
    scaler.fit(train[columns_to_scale])
    
    # apply the scaler
    train_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(train[columns_to_scale]),
                                                  columns=train[columns_to_scale].columns.values).set_index([train.index.values])
                                                  
    validate_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(validate[columns_to_scale]),
                                                  columns=validate[columns_to_scale].columns.values).set_index([validate.index.values])
    
    test_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(test[columns_to_scale]),
                                                 columns=test[columns_to_scale].columns.values).set_index([test.index.values])
    
    if return_scaler:
        return scaler, train_scaled, validate_scaled, test_scaled
    else:
        return train_scaled, validate_scaled, test_scaled
    


In [45]:
train_scaled, validate_scaled, test_scaled= scale_data(train, validate, test, 
               columns_to_scale= ['age', 'annual_income'],
               return_scaler=False)
train_scaled.shape, validate_scaled.shape, test_scaled.shape

((112, 5), (48, 5), (40, 5))

In [46]:
# # peak into scaled train data
train_scaled.head()

,customer_id,age,annual_income,spending_score,Male
130,131,0.557692,0.459016,9,1
62,63,0.942308,0.262295,52,0
13,14,0.115385,0.040984,77,0
173,174,0.346154,0.590164,92,1
144,145,0.134615,0.508197,12,1
